<a href="https://colab.research.google.com/github/pisceno/AI-Generative/blob/main/regresion_02_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regresion multilineal

In [ ]:
#pasos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df= pd.read_csv("https://raw.githubusercontent.com/GerhardSpross/clasificacion-prueba/refs/heads/main/day.csv")
display(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


None

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
5,6,2011-01-06,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.089565,88,1518,1606
6,7,2011-01-07,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.168726,148,1362,1510
7,8,2011-01-08,1,0,1,0,6,0,2,0.165000,0.162254,0.535833,0.266804,68,891,959
8,9,2011-01-09,1,0,1,0,0,0,1,0.138333,0.116175,0.434167,0.361950,54,768,822
9,10,2011-01-10,1,0,1,0,1,1,1,0.150833,0.150888,0.482917,0.223267,41,1280,1321


In [ ]:
def z(x, w, j, propiedades):
    sumaz = sum([w[p] * x[p].iloc[j] for p in propiedades])
    return w["sesgo"] + sumaz

def gradiente(x, y, yp, propiedades):
    grad = {}
    grad["sesgo"] = sum([(y.iloc[j] - yp[j]) * (-1) for j in range(0, len(y))])

    #grad["sesgo"] = sum ([yp[j]*(1-yp[j])*(y[j])-yp[j]*(-1) for j in range(0,len(y))])

    for p in propiedades:
        grad[p] = sum([(y.iloc[j] - yp[j]) * (-x[p].iloc[j]) for j in range(0, len(y))])

        #grad[p] = sum([yp[j]*(1-yp[j])*(y[j]-yp[j])*(-x[p][j]) for j in range(0,len(y))])
    return grad

#faltaban estas dos lineas, creo que si van aqui, G
#def active(zp):
  #return 1.0/(1.0+np.exp(-zp))

#df = pd.read_csv("day.csv")

# Se Elimina la columna no numéricas o irrelevantes (instant y dteday)
# y la columna objetivo (cnt) antes de definir las propiedades.
columnas_a_eliminar = ['instant', 'dteday','casual', 'registered']
df_recortado = df.drop(columns=columnas_a_eliminar)

#Almacena los nombres de las columnas que son características
#(todo menos la última columna, que es el valor a predecir, cnt).
df.info()
propiedades = df_recortado.columns[:-1]


#Crea un diccionario w para guardar el sesgo y todos los pesos del modelo.
w = {}

w["sesgo"] = 1
#Inicializa todos los pesos ($w_1, w_2,..$) con el valor 0.
# El algoritmo de Descenso del Gradiente encontrará los valores
#  correctos a partir de estos puntos de partida.

for p in propiedades:
    w[p] = 0

##==========================Transformación de datos===========================
x = df_recortado[propiedades]
#Guarda la variable objetivo (cnt) en una variable llamada y
y = df_recortado[df_recortado.columns[-1]]

# Initialize the MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit and transform the features (x) and the target variable (y)
x_scaled = scaler_x.fit_transform(x)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Convert the scaled arrays back to pandas DataFrames/Series if needed
x = pd.DataFrame(x_scaled, columns=propiedades)
y = pd.Series(y_scaled.flatten())


# Almacena los rangos de normalización para poder desnormalizar después
# min_max_norm = {}

# Normaliza la variable objetivo (y)
# y_norm, y_min, y_max = normalizar_columna(y)
# y = y_norm.tolist() # Usaremos la 'y' normalizada en el entrenamiento

#min_max_norm['y'] = {'min': y_min, 'max': y_max}
# Almacena los rangos de normalización para referencia futura (desnormalización)
# min_max_norm = {'y': {'min': y_min, 'max': y_max}}

#for p in propiedades:
    #Normalizaremos en un rango [0, 1]
#    col_norm, min_val, max_val = normalizar_columna(x[p])
#    x[p] = col_norm
#    min_max_norm[p] = {'min': min_val, 'max': max_val}


yp = [0] * len(y)
alpha = 0.0001
#alpha = 0.01

for k in range(500):

    for j in range(0, len(y)):
        yp[j] = z(x, w, j, propiedades)

    grad = gradiente(x, y, yp, propiedades)
    for p in propiedades:
        w[p] = w[p] - alpha * grad[p]
    w["sesgo"] = w["sesgo"] - alpha * grad["sesgo"]
    error = sum([(y[i] - yp[i]) ** 2 for i in range(0, len(y))])
    print(error)


# print(np.array(y))
# print(np.round(np.array(yp), 2))

# Inverse transform the predictions back to original scale
yp_original_scale = scaler_y.inverse_transform(np.array(yp).reshape(-1, 1)).flatten()

print("Original y values:", scaler_y.inverse_transform(y.values.reshape(-1, 1)).flatten())
print("Predicted yp values (original scale):", np.round(yp_original_scale, 2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB
207.72370625701907
136.69406935391729
98.72224888916558
78.23509368404841
67.00015933799527
6

In [ ]:
# Mostrar los resultados del entrenamiento
print("Resultados de la regresión multilineal:")
print("======================================")

# Mostrar el sesgo (w0)
print(f"El sesgo (w0) es: {w['sesgo']:.4f}")

# Mostrar los pesos (w1, w2, ..., wk) para cada propiedad
print("\nPesos (w) para cada característica:")
for p in propiedades:
    print(f"  w para '{p}': {w[p]:.4f}")

# Mostrar el error cuadrático medio final (E)
print(f"\nEl error cuadrático medio final (E) es: {error:.4f}")

print("\nInterpretación:")
print("--------------")
print("La ecuación de la regresión multilineal es de la forma:")
print("y_predicha = w0 + w1*x1 + w2*x2 + ... + wk*xk")
print("donde:")
print("w0 es el sesgo (intercepto).")
print("w1, w2, ..., wk son los pesos para cada característica (x1, x2, ..., xk).")
print("El error (E) indica qué tan bien se ajusta el modelo a los datos (un valor más bajo es mejor).")

Resultados de la regresión multilineal:
El sesgo (w0) es: 0.4059

Pesos (w) para cada característica:
  w para 'season': 0.1663
  w para 'yr': 0.2222
  w para 'mnth': -0.0335
  w para 'holiday': -0.0783
  w para 'weekday': 0.0142
  w para 'workingday': 0.0011
  w para 'weathersit': -0.1179
  w para 'temp': 0.2191
  w para 'atemp': 0.2008
  w para 'hum': -0.2818
  w para 'windspeed': -0.2528

El error cuadrático medio final (E) es: 7.9953

Interpretación:
--------------
La ecuación de la regresión multilineal es de la forma:
y_predicha = w0 + w1*x1 + w2*x2 + ... + wk*xk
donde:
w0 es el sesgo (intercepto).
w1, w2, ..., wk son los pesos para cada característica (x1, x2, ..., xk).
El error (E) indica qué tan bien se ajusta el modelo a los datos (un valor más bajo es mejor).


Archivo hour.csv

In [ ]:
df1 = pd.read_csv("https://raw.githubusercontent.com/GerhardSpross/clasificacion-prueba/refs/heads/main/hour.csv")
display(df.info())
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     731 non-null    int64  
 1   dteday      731 non-null    object 
 2   season      731 non-null    int64  
 3   yr          731 non-null    int64  
 4   mnth        731 non-null    int64  
 5   holiday     731 non-null    int64  
 6   weekday     731 non-null    int64  
 7   workingday  731 non-null    int64  
 8   weathersit  731 non-null    int64  
 9   temp        731 non-null    float64
 10  atemp       731 non-null    float64
 11  hum         731 non-null    float64
 12  windspeed   731 non-null    float64
 13  casual      731 non-null    int64  
 14  registered  731 non-null    int64  
 15  cnt         731 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.5+ KB


None

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
5,6,2011-01-06,1,0,1,0,4,1,1,0.204348,0.233209,0.518261,0.089565,88,1518,1606
6,7,2011-01-07,1,0,1,0,5,1,2,0.196522,0.208839,0.498696,0.168726,148,1362,1510
7,8,2011-01-08,1,0,1,0,6,0,2,0.165000,0.162254,0.535833,0.266804,68,891,959
8,9,2011-01-09,1,0,1,0,0,0,1,0.138333,0.116175,0.434167,0.361950,54,768,822
9,10,2011-01-10,1,0,1,0,1,1,1,0.150833,0.150888,0.482917,0.223267,41,1280,1321
